In [1]:
import numpy as np 
import pandas as pd 
import re
from random import randint

maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000
numStories = 63000

In [2]:
fake_df = pd.read_csv('fake.csv')
print fake_df.columns
fake_text = fake_df['text'].values

Index([u'uuid', u'ord_in_thread', u'author', u'published', u'title', u'text',
       u'language', u'crawled', u'site_url', u'country', u'domain_rank',
       u'thread_title', u'spam_score', u'main_img_url', u'replies_count',
       u'participants_count', u'likes', u'comments', u'shares', u'type'],
      dtype='object')


In [3]:
real_df = pd.read_json('signalmedia-1m.jsonl',lines=True)
print real_df.columns
real_text = real_df['content'].values

Index([u'content', u'id', u'media-type', u'published', u'source', u'title'], dtype='object')


In [4]:
# choose 50,000 random stories from the real dataset
real_i = np.random.choice(len(real_text), size=50000, replace=False)
real_text = real_text[real_i]

In [5]:
def loadGloveModel():
    print "Loading Glove Model"
    f = open('glove.42B.300d.txt','r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

In [6]:
# glove word-vec dictionary
glove = loadGloveModel()

Loading Glove Model


KeyboardInterrupt: 

In [5]:
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [17]:
# stories x the first 200 words
train_matrix = np.empty((numStories, maxSeqLength+1), dtype='object')
story_index = 0
for story in real_text:
    words = np.empty((maxSeqLength+1), dtype='object')
    indexCounter = 0
    cleanedLine = cleanSentences(story)
    split = cleanedLine.split()
    if len(split)< maxSeqLength:
        indexCounter = maxSeqLength-len(split)
    else:
        split = split[:maxSeqLength]
    for word in split:
        words[indexCounter] = word
        indexCounter = indexCounter + 1
    words[maxSeqLength]="REAL"
    train_matrix[story_index] = words
    story_index+=1

for story in fake_text:
    words = np.empty((maxSeqLength+1), dtype='object')
    indexCounter = 0
    if type(story) is float:
        story = str(story)
    cleanedLine = cleanSentences(story)
    split = cleanedLine.split()
    if len(split)< maxSeqLength:
        indexCounter = maxSeqLength-len(split)
    else:
        split = split[:maxSeqLength]
    for word in split:
        words[indexCounter] = word
        indexCounter = indexCounter + 1
    words[maxSeqLength]="Fake"
    train_matrix[story_index] = words
    story_index+=1

np.save('train_matrix', train_matrix)